In [92]:
import numpy as np
import pandas as pd
import pybaseball
import seaborn as sns
import matplotlib.pyplot as plot
from deepdiff import DeepDiff
from pybaseball import bwar_pitch
from pybaseball import bwar_bat
from pybaseball import cache
from pybaseball.lahman import *
from pybaseball import chadwick_register
from pybaseball import pitching_stats
from pybaseball import batting_stats

cache.enable()


In [93]:
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 100)

In [94]:
fbat = pd.DataFrame(pybaseball.batting_stats(start_season=1921, end_season=2021))

In [95]:
fpit = pd.DataFrame(pybaseball.pitching_stats(start_season=1921, end_season=2021))

In [96]:
ftpit = pd.DataFrame(pybaseball.team_pitching(start_season=1921, end_season=2021))

In [97]:
ftbat = pd.DataFrame(pybaseball.team_batting(start_season=1921, end_season=2021))

In [98]:
ftfld = pd.DataFrame(pybaseball.team_fielding(start_season=1921, end_season=2021))

In [99]:
# bref pitching WAR
bwar_pitch = pd.DataFrame(bwar_pitch(return_all=True))

# bref batting WAR
bwar_bat = pd.DataFrame(bwar_bat(return_all=True))

In [100]:
people = pd.DataFrame(people())
appearances = pd.DataFrame(appearances())

In [101]:
chadwick = chadwick_register(save=True)
rosetta = pd.DataFrame(chadwick) 


----
^^^^^ Start over from here ^^^^^^

In [102]:
people.sample(5)

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
12881,morelmi01,1985.0,9.0,6.0,USA,MS,Amory,NaN,NaN,NaN,NaN,NaN,NaN,Mitch,Moreland,Mitchell Austin,245.0,75.0,L,L,2010-07-29,2021-08-26,morem001,morelmi01
19747,whitemi01,1862.0,NaN,NaN,CAN,ON,Toronto,1901.0,8.0,15.0,USA,CA,Highland,Milt,Whitehead,Milton Pringle,154.0,68.0,B,R,1884-04-20,1884-10-19,whitm102,whitemi01
15587,riverma01,1969.0,11.0,29.0,Panama,Panama,Panama,NaN,NaN,NaN,NaN,NaN,NaN,Mariano,Rivera,Mariano,195.0,74.0,R,R,1995-05-23,2013-09-26,rivem002,riverma01
18562,tippeji01,1849.0,6.0,18.0,USA,CT,Middletown,1895.0,4.0,21.0,USA,CT,New Haven,Jim,Tipper,James,148.0,65.0,NaN,NaN,1872-04-26,1875-10-28,tippj101,tippeji01
3682,coombda01,1942.0,3.0,23.0,USA,ME,Lincoln,NaN,NaN,NaN,NaN,NaN,NaN,Danny,Coombs,Daniel Bernard,200.0,76.0,R,L,1963-09-27,1971-07-23,coomd101,coombda01


In [103]:
rosetta = rosetta.rename(columns={
    "key_fangraphs": "IDfg"
})

In [104]:
ftbat.sample()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
3822,1209,2000,Ben Grieve,OAK,24,158,594,675,166,98,40,1,27,92,104,73,2.0,130,3,5.0,0,32.0,3,0.0,0.279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.108,0.193,0.56,0.359,0.487,0.845,0.207,0.314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.365,14.3,103,13.4,-7.0,21.8,-7.3,24.1,2.2,NaN,2.9,115,-0.24,-12.36,12.12,9.69,0.88,0.98,1.14,2.0,0.71,-0.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-14.3,0.7,NaN,24 - 24,14.1,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101,113,123,103,109,123,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [105]:
def drop_y(df):
    # list comprehension of the cols that end with '_y'
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)


In [106]:
# merge magic
result = pd.merge(ftbat, rosetta,  how='inner', left_on=['IDfg'], right_on=['IDfg'], suffixes=('', '_y'))
drop_y(result)

In [107]:
result = pd.merge(result, fpit,  how='inner', left_on=['IDfg'], right_on=['IDfg'], suffixes=('', '_y'))
drop_y(result)

In [108]:
result = pd.merge(result, ftbat,  how='inner', left_on=['IDfg'], right_on=['IDfg'], suffixes=('', '_y'))
drop_y(result)

In [109]:
result.sample()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,name_last,name_first,key_mlbam,key_retro,key_bbref,mlb_played_first,mlb_played_last
371151,1000450,1979,Dusty Baker,LAD,30,151,554,616,152,99,29,1,23,86,88,56,0.0,70,1,3.0,2,16.0,11,4.0,0.274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.091,0.114,0.8,0.34,0.455,0.795,0.181,0.278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.354,14.6,87,15.4,-1.0,18.4,-7.1,28.7,2.9,NaN,4.0,121,1.5,-11.62,13.12,13.87,1.61,1.09,2.86,4.0,1.05,0.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.1,0.8,NaN,30 - 30,16.1,2.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102,104,93,102,114,139,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Baker,Dusty,110481,baked002,bakerdu01,1968.0,1986.0


In [110]:
result = pd.merge(result, people,  how='inner', left_on=['key_bbref'], right_on=['bbrefID'], suffixes=('', '_y'))
drop_y(result)

In [111]:
everyone = result

In [112]:
everyone.sample(50)  # THIS WORKS DON'T TOUCH ANYTHING ABOVE HERE OR I WILL SCREAM

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,name_last,name_first,key_mlbam,key_retro,key_bbref,mlb_played_first,mlb_played_last,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
411163,1011850,1929,Earl Sheely,PIT,36,139,485,585,142,110,22,4,6,63,88,75,NaN,24,4,NaN,21,NaN,6,NaN,0.293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.128,0.041,3.13,0.392,0.392,0.784,0.099,0.299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.369,8.1,86,-1.2,0.0,19.1,-6.0,13.9,1.3,NaN,3.8,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.0,0.9,NaN,36 - 36,-0.3,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96,151,66,107,88,71,95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,

In [320]:
# Constants
KG_TO_LB = 0.453592
M_TO_IN = 0.0254


In [322]:

### Read infinity baseball data to DataFrames
# a table of all player biographical info and ids


# each player's games played per position for each season
appearances = pd.DataFrame(appearances())

# batting stats by year, regular season
batting = pd.DataFrame(batting())

# batting stats by year, post season
batting_post = pd.DataFrame(batting_post())

# fielding stats by year
fielding = pd.DataFrame(fielding())

# games played in left, center, right field
fielding_of = pd.DataFrame(fielding_of())

# LF/CF/RF splits
fielding_of_split = pd.DataFrame(fielding_of_split())

# postseason fielding
fielding_post = pd.DataFrame(fielding_post())

# home game attendance by park by year
home_games = pd.DataFrame(home_games())

# historical player pitching stats
pitching = pd.DataFrame(pitching())

# postseason pitching stats
pitching_post = pd.DataFrame(pitching_post())

# playoff series winners and losers
series_post = pd.DataFrame(series_post())

# data on teams by year: record, division, stadium, attendance, etc
teams = pd.DataFrame(teams())

# current and historical franchises, whether they're still active, and their ids
teams_franchises = pd.DataFrame(teams_franchises())

# split season data for teams
teams_half = pd.DataFrame(teams_half())



/Users/bean/.pyenv/versions/3.10.3/lib/python3.10/site-packages/pybaseball/league_batting_stats.py:64: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  table = table.drop('', 1)


In [323]:
### Add BMI Calcs
# BMI Calculations
people["KG"] = people["weight"] * KG_TO_LB
people["meters"] = people["height"] * M_TO_IN
people["BMI"] = people["KG"] / people["meters"] ** 2
people["ratio"] = people["meters"] * people["BMI"]


## Assemble Monster DataFrame of Everything About Player Ever


In [324]:
# Let's change some of these column names to save ourselves some merging hassle
chadwick = chadwick.rename(
    columns={
        "name_last": "nameLast",
        "name_first": "nameFirst",
        "key_fangraphs": "fangraphsID",
        "key_bbref": "playerID",
        "key_retro": "retroID",
        "key_mlbam": "mlbID",
    }
)

# Merge chadwick into people
df = people.merge(
    chadwick,
    left_on=["playerID", "retroID", "nameLast", "nameFirst"],
    right_on=["playerID", "retroID", "nameLast", "nameFirst"],
    how="outer",
)

In [325]:
chadwick

,nameLast,nameFirst,mlbID,retroID,playerID,fangraphsID,mlb_played_first,mlb_played_last
0,Aardsma,David,430911,aardd001,aardsda01,1902,2004.0,2015.0
1,Aaron,Hank,110001,aaroh101,aaronha01,1000001,1954.0,1976.0
2,Aaron,Tommie,110002,aarot101,aaronto01,1000002,1962.0,1971.0
3,Aase,Don,110003,aased001,aasedo01,1000003,1977.0,1990.0
4,Abad,Andy,407577,abada001,abadan01,506,2001.0,2006.0
...,...,...,...,...,...,...,...,...
24253,Zupo,Frank,124788,zupof101,zupofr01,1014464,1957.0,1961.0
24254,Zuvella,Paul,124789,zuvep001,zuvelpa01,1014465,1982.0,1991.0
24255,Zuverink,George,124790,zuveg101,zuverge01,1014466,1951.0,1959.0
24256,Zwilling,Dutch,124791,zwild101,zwilldu01,1014467,1910.0,1916.0


In [326]:
bwar_bat.columns

Index(['name_common', 'age', 'mlb_ID', 'player_ID', 'year_ID', 'team_ID',
       'stint_ID', 'lg_ID', 'PA', 'G', 'Inn', 'runs_bat', 'runs_br', 'runs_dp',
       'runs_field', 'runs_infield', 'runs_outfield', 'runs_catcher',
       'runs_good_plays', 'runs_defense', 'runs_position', 'runs_position_p',
       'runs_replacement', 'runs_above_rep', 'runs_above_avg',
       'runs_above_avg_off', 'runs_above_avg_def', 'WAA', 'WAA_off', 'WAA_def',
       'WAR', 'WAR_def', 'WAR_off', 'WAR_rep', 'salary', 'pitcher', 'teamRpG',
       'oppRpG', 'oppRpPA_rep', 'oppRpG_rep', 'pyth_exponent',
       'pyth_exponent_rep', 'waa_win_perc', 'waa_win_perc_off',
       'waa_win_perc_def', 'waa_win_perc_rep', 'OPS_plus', 'TOB_lg', 'TB_lg'],
      dtype='object')

In [327]:
### Add BWAR Stats
#### Add BWAR Batting
##### BWAR Batting Prep
bwar_bat.columns
bwar_bat = bwar_bat.rename(columns={
    'player_ID': 'playerID',
    'mlb_ID': 'mlbID',
    'stint_ID': 'stint'
})

##### BWAR Batting Merge
df = df.merge(
    bwar_bat, left_on=["playerID", "mlbID"], right_on=["playerID", "mlbID"], how="outer"
)


In [328]:
df

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,KG,meters,BMI,ratio,mlbID,fangraphsID,mlb_played_first,mlb_played_last,name_common,age,year_ID,team_ID,stint,lg_ID,PA,G,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_rep,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAA,WAA_off,WAA_def,WAR,WAR_def,WAR_off,WAR_rep,salary,pitcher,teamRpG,oppRpG,oppRpPA_rep,oppRpG_rep,pyth_exponent,pyth_exponent_rep,waa_win_perc,waa_win_perc_off,waa_win_perc_def,waa_win_perc_rep,OPS_plus,TOB_lg,TB_lg
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,97.52228,1.905,26.87286,51.192798,430911.0,1902.0,2004.0,2015.0,David Aardsma,22.0,2004.0,SFG,1.0,NL,0.0,11.0,10.7,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,-0.01,0.00,-0.01,0.00,0.00,300000.0,Y,4.67092,4.67092,0.08651,4.67092,1.890,1.890,0.5000,0.5000,0.5000,0.5000,NaN,0.000,0.000
1,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,97.52228,1.905,26.87286,51.192798,430911.0,1902.0,2004.0,2015.0,David Aardsma,24.0,2006.0,CHC,1.0,NL,3.0,43.0,53.0,-0.90,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.46,0.00,-0.4,-0.4,-0.4,0.0,-0.04,-0.04,-0.01,-0.04,-0.01,-0.04,0.00,NaN,Y,4.85675,4.86675,0.09085,4.86457,1.912,1.913,0.4990,0.4990,0.5000,0.4998,-100.000000,0.694,0.896
2,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,97.52228,1.905,26.87286,51.192798,430911.0,1902.0,2004.0,2015.0,David Aardsma,25.0,2007.0,CHW,1.0,AL,0.0,2.0,32.3,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,387500.0,Y,4.85895,4.85895,0.08422,4.85895,1.912,1.912,0.5000,0.5000,0.5000,0.5000,NaN,0.000,0.000
3,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,97.52228,1.905,26.87286,51.192798,430911.0,1902.0,2004.0,2015.0,David Aardsma,26.0,2008.0,BOS,1.0,AL,1.0,5.0,48.7,-0.29,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.14,0.00,-0.2,-0.2,-0.2,0.0,-0.02,-0.02,0.00,-0.02,0.00,-0.02,0.00,403250.0,Y,4.67400,4.70400,0.08092,4.69650,1.893,1.894,0.4970,0.4970,0.5000,0.4992,-100.000000,0.345,0.434
4,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,97.52228,1.905,26.87286,51.192798,430911.0,1902.0,2004.0,2015.0,David Aardsma,27.0,2009.0,SEA,1.0,AL,0.0,3.0,71.3,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,419000.0,Y,4.79788,4.79788,0.08302,4.79788,1.905,1.905,0.5000,0.5000,0.5000,0.5000,NaN,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124071,zomphch01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Charles Zomphier,21.0,1927.0,SLS,2.0,NNL,19.0,10.0,NaN,-0.13,0.00,0.0,0.0,NaN,NaN,NaN,NaN,0.0,-0.01,0.00,0.65,0.5,-0.1,-0.1,0.0,-0.01,-0.01,0.00,0.05,0.00,0.05,0.06,NaN,N,4.90709,4.92109,0.09081,4.85616,1.918,1.915,0.4986,0.4986,0.4999,0.4936,91.320701,6.492,7.186
124072,zomphch01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [329]:
#### Add BWAR Pitching
##### BWAR Pitching Prep

# merge bwar_pitch, check
bwar_pitch = bwar_pitch.rename(
    columns={"stint_ID": "stint", "mlb_ID": "mlbID", "player_ID": "playerID"}
)

##### BWAR Pitching Merge
print(df.columns.tolist())
df = df.merge(
    bwar_pitch,
    left_on=[
        "G",
        "WAA",
        "WAR",
        "WAR_rep",
        "age",
        "lg_ID",
        "mlbID",
        "oppRpG",
        "oppRpG_rep",
        "playerID",
        "pyth_exponent",
        "pyth_exponent_rep",
        "runs_above_avg",
        "runs_above_rep",
        "salary",
        "stint",
        "teamRpG",
        "team_ID",
        "waa_win_perc",
        "waa_win_perc_rep",
        "year_ID",
    ],
    right_on=[
        "G",
        "WAA",
        "WAR",
        "WAR_rep",
        "age",
        "lg_ID",
        "mlbID",
        "oppRpG",
        "oppRpG_rep",
        "playerID",
        "pyth_exponent",
        "pyth_exponent_rep",
        "runs_above_avg",
        "runs_above_rep",
        "salary",
        "stint",
        "teamRpG",
        "team_ID",
        "waa_win_perc",
        "waa_win_perc_rep",
        "year_ID",
    ],
    how="outer",
)

['playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry', 'birthState', 'birthCity', 'deathYear', 'deathMonth', 'deathDay', 'deathCountry', 'deathState', 'deathCity', 'nameFirst', 'nameLast', 'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame', 'retroID', 'bbrefID', 'KG', 'meters', 'BMI', 'ratio', 'mlbID', 'fangraphsID', 'mlb_played_first', 'mlb_played_last', 'name_common', 'age', 'year_ID', 'team_ID', 'stint', 'lg_ID', 'PA', 'G', 'Inn', 'runs_bat', 'runs_br', 'runs_dp', 'runs_field', 'runs_infield', 'runs_outfield', 'runs_catcher', 'runs_good_plays', 'runs_defense', 'runs_position', 'runs_position_p', 'runs_replacement', 'runs_above_rep', 'runs_above_avg', 'runs_above_avg_off', 'runs_above_avg_def', 'WAA', 'WAA_off', 'WAA_def', 'WAR', 'WAR_def', 'WAR_off', 'WAR_rep', 'salary', 'pitcher', 'teamRpG', 'oppRpG', 'oppRpPA_rep', 'oppRpG_rep', 'pyth_exponent', 'pyth_exponent_rep', 'waa_win_perc', 'waa_win_perc_off', 'waa_win_perc_def', 'waa_win_perc_rep', 'OPS_p

In [330]:
bwar_pitch.sample(10)

,name_common,age,mlbID,playerID,year_ID,team_ID,stint,lg_ID,G,GS,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg,runs_above_avg_adj,runs_above_rep,RpO_replacement,GR_leverage_index_avg,WAR,salary,teamRpG,oppRpG,pyth_exponent,waa_win_perc,WAA,WAA_adj,oppRpG_rep,pyth_exponent_rep,waa_win_perc_rep,WAR_rep,ERA_plus,ER_lg
21875,Tanner Houck,24.0,656557.0,houckta01,2020,BOS,1,AL,3,3,51,51.0,0.0,2,10.612,0.535,0.0,0.020,108,108.857,12.113,32.0,0.0204,1.0,10.113,9.986,11.806,0.211,1.0000,1.29,NaN,4.77556,1.44689,1.684,0.8819,1.1457,-0.0216,5.34455,1.934,0.4458,0.1628,919.700000,9.197
32433,Vinegar Bend Mizell,26.0,119208.0,mizelvi01,1957,STL,1,NL,33,21,448,379.0,69.0,69,71.824,0.000,NaN,2.719,102,101.681,70.267,471.0,0.1026,26.5,1.267,2.658,16.293,0.195,1.4367,1.74,NaN,4.33763,4.25708,1.846,0.5086,0.2838,-0.0647,4.79581,1.878,0.4530,1.5222,106.143548,65.809
49457,Jerry Walker,25.0,123830.0,walkeje01,1964,CLE,1,AL,6,0,29,0.0,29.0,5,4.630,-0.191,NaN,-0.361,100,100.103,4.805,29.0,0.0065,-55.6,-0.195,-0.065,0.531,0.176,0.8067,0.05,NaN,4.05437,4.06520,1.816,0.4988,-0.0072,-0.0250,4.17655,1.823,0.4865,0.0828,81.280000,4.064
32578,Sid Monge,28.0,119249.0,mongesi01,1979,CLE,1,AL,76,0,393,0.0,393.0,37,68.324,-5.657,NaN,-2.880,101,100.886,66.128,361.0,0.0760,-37.9,29.128,29.163,42.675,0.209,1.9767,5.57,NaN,4.70696,4.32324,1.872,0.5397,3.0172,1.2034,4.88288,1.905,0.4825,1.3485,178.191429,62.367
31981,Percy Miller,40.0,NaN,millepe01,1937,DTS,0,NAL,9,8,163,NaN,NaN,51,33.115,0.000,NaN,-2.054,100,NaN,35.169,199.0,0.3210,-6.4,-15.831,-12.738,-9.308,0.254,1.0000,-0.61,NaN,5.75633,7.17166,2.074,0.3879,-1.0089,-0.1393,6.48096,2.042,0.4398,0.5350,67.840000,27.136
154,Juan Acevedo,30.0,110032.0,aceveju01,2000,MIL,1,NL,62,0,248,0.0,248.0,38,46.975,-3.598,NaN,1.638,98,98.772,41.226,264.0,0.0557,29.4,3.226,2.982,11.095,0.219,1.0356,0.86,612500.0,5.05048,5.00238,1.930,0.5046,0.2852,-0.1619,5.17548,1.940,0.4881,0.7320,119.748571,41.912
39935,John Riedling,27.0,150180.0,riedljo01,2003,CIN,1,NL,55,8,303,116.0,187.0,61,54.597,-1.721,NaN,0.259,96,96.629,50.843,340.0,0.0701,3.7,-10.157,-10.637,-1.355,0.201,1.0196,-0.34,350000.0,4.62384,4.81724,1.896,0.4806,-1.0670,-0.1596,4.78421,1.894,0.4839,0.8845,84.265455,46.346
40671,Ed Roebuck,33.0,121371.0,roebued01,1965,PHI,1,NL,44,0,151,0.0,151.0,27,23.292,-0.990,NaN,-1.571,97,98.459,23.505,176.0,0.0385,-40.8,-3.495,-3.537,1.195,0.180,1.1593,0.02,NaN,4.01644,4.09683,1.816,0.4910,-0.3960,-0.1207,4.12176,1.818,0.4882,0.5342,101.942105,19.369
38687,Pat Ragan,30.0,120885.0,raganpa01,1914,BRO,1,NL,38,25,636,562.0,74.0,104,90.188,0.000,NaN,-1.542,102,101.729,93.316,706.0,0.1542,-10.0,-10.684,-9.643,4.504,0.167,1.0000,0.57,NaN,3.84670,4.10046,1.805,0.4712,-1.0944,-0.1124,4.24838,1.815,0.4551,1.7799,95.542029,65.924
41409,Red Ruffing,30.0,121547.0,ruffire01,1935,NYY,1,AL,30,29,666,653.0,13.0,88,122.830,0.000,NaN,7.844,91,91.213,104.882,751.0,0.1669,47.0,16.882,17.171,43.045,0.229,1.8600,3.82,12000.0,5.10508,4.53272,1.907,0.5565,1.6950,-0.1410,5.97577,1.985,0.4225,2.2655,130.092208,100.171


In [331]:
#### Add Fielding_OF
##### Fielding_OF Prep
fielding_of.columns
fielding_of = fielding_of.rename(columns={"yearID": "year_ID"})
##### Fielding_OF Merge
df = df.merge(
    fielding_of,
    left_on=["playerID", "year_ID", "stint"],
    right_on=["playerID", "year_ID", "stint"],
    how="left",
)

In [332]:
df

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,KG,meters,BMI,ratio,mlbID,fangraphsID,mlb_played_first,mlb_played_last,name_common_x,age,year_ID,team_ID,stint,lg_ID,PA,G,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_rep,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAA,WAA_off,WAA_def,WAR,WAR_def,WAR_off,WAR_rep,salary,pitcher,teamRpG,oppRpG,oppRpPA_rep,oppRpG_rep,pyth_exponent,pyth_exponent_rep,waa_win_perc,waa_win_perc_off,waa_win_perc_def,waa_win_perc_rep,OPS_plus,TOB_lg,TB_lg,name_common_y,GS,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg_adj,RpO_replacement,GR_leverage_index_avg,WAA_adj,ERA_plus,ER_lg,Glf,Gcf,Grf
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,97.52228,1.905,26.87286,51.192798,430911.0,1902.0,2004.0,2015.0,David Aardsma,22.0,2004.0,SFG,1.0,NL,0.0,11.0,10.7,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000,0.000,0.0,0.0,0.0000,0.00,-0.01,0.00,-0.01,0.00,0.0000,300000.0,Y,4.67092,4.67092,0.08651,4.67092,1.890,1.890,0.5000,0.500,0.5,0.5000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,97.52228,1.905,26.87286,51.192798,430911.0,1902.0,2004.0,2015.0,David Aardsma,24.0,2006.0,CHC,1.0,NL,3.0,43.0,53.0,-0.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.46,0.0,-0.400,-0.400,-0.4,0.0,-0.0400,-0.04,-0.01,-0.04,-0.01,-0.04,0.0000,NaN,Y,4.85675,4.86675,0.09085,4.86457,1.912,1.913,0.4990,0.499,0.5,0.4998,-100.0,0.694,0.896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,97.52228,1.905,26.87286,51.192798,430911.0,1902.0,2004.0,2015.0,David Aardsma,25.0,2007.0,CHW,1.0,AL,0.0,2.0,32.3,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000,0.000,0.0,0.0,0.0000,0.00,0.00,0.00,0.00,0.00,0.0000,387500.0,Y,4.85895,4.85895,0.08422,4.85895,1.912,1.912,0.5000,0.500,0.5,0.5000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,97.52228,1.905,26.87286,51.192798,430911.0,1902.0,2004.0,2015.0,David Aardsma,26.0,2008.0,BOS,1.0,AL,1.0,5.0,48.7,-0.29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.14,0.0,-0.200,-0.200,-0.2,0.0,-0.0200,-0.02,0.00,-0.02,0.00,-0.02,0.0000,403250.0,Y,4.67400,4.70400,0.08092,4.69650,1.893,1.894,0.4970,0.497,0.5,0.4992,-100.0,0.345,0.434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,97.52228,1.905,26.87286,51.192798,430911.0,1902.0,2004.0,2015.0,David Aardsma,27.0,2009.0,SEA,1.0,AL,0.0,3.0,71.3,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000,0.000,0.0,0.0,0.0000,0.00,0.00,0.00,0.00,0.00,0.0000,419000.0,Y,4.79788,4.79788,0.08302,4.79788,1.905,1.905,0.5000,0.500,0.5,0.5000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [333]:
#### Add Fielding
##### Fielding Check
fielding.columns
fielding = fielding.rename(columns={"yearID": "year_ID"})
##### Fielding Merge
df = df.merge(
    fielding,
    left_on=["playerID", "year_ID", "G", "stint"],
    right_on=["playerID", "year_ID", "G", "stint"],
    how="left",
)

In [336]:
### Meet the Monster
df.groupby('POS')

birthYear                                                           \
        count         mean        std     min     25%     50%      75%   
POS                                                                      
1B     1170.0  1902.439316  41.638575  1840.0  1868.0  1893.0  1924.75   
2B     1348.0  1910.718101  44.094965  1840.0  1869.0  1904.0  1950.00   
3B     1282.0  1910.498440  44.101512  1840.0  1870.0  1900.0  1951.00   
C      2698.0  1928.071164  43.448725  1840.0  1889.0  1937.0  1967.00   
OF     3462.0  1905.241190  46.429684  1832.0  1865.0  1889.0  1949.00   
P     23797.0  1929.027735  35.108489  1841.0  1900.0  1930.0  1957.00   
SS     1775.0  1919.214085  43.450646  1836.0  1881.5  1917.0  1960.00   

            birthMonth                                                 \
        max      count      mean       std  min  25%  50%   75%   max   
POS                                                                     
1B   1996.0     1150.0  6.639130  3.340971  1.0  4.0  7.0   9.0  12.0   
2B   1997.0     1332.0  6.720721  3.392008  1.0  4.0  7.0  10.0  12.0   
3B   1998.0     1258.0  6.717806  3.492784  1.0  3.0  7.0  10.0  12.0   
C    1999.0     2675.0  6.682617  3.380592  1.0  4.0  7.0  10.0  12.0   
OF   1999.0     3406.0  6.448620  3.453765  1.0  3.0  7.0  10.0  12.0   
P    1999.0    23762.0  6.617709  3.436383  1.0  4.0  7.0  10.0  12.0   
SS   1998.0     1760.0  6.469318  3.627037  1.0  3.0  7.0  10.0  12.0   

    birthDay                                                  deathYear  \
       count       mean       std  min  25%   50%   75%   max     count   
POS                                                                       
1B    1131.0  16.010610  8.581410  1.0  9.0  17.0  23.0  31.0     914.0   
2B    1309.0  14.923606  8.800992  1.0  7.0  15.0  22.0  31.0     917.0   
3B    1245.0  15.978313  8.283192  1.0  9.0  16.0  23.0  31.0     891.0   
C     2647.0  15.489989  8.359720  1.0  8.0  15.0  22.0  31.0    1414.0   
OF    3362.0  15.506841  8.624335  1.0  9.0  15.0  23.0  31.0    2474.0   
P    23707.0  15.707513  8.825664  1.0  8.0  16.0  23.0  31.0   14294.0   
SS    1748.0  15.560641  8.711511  1.0  8.0  16.0  23.0  31.0    1097.0   

                                                                     \
            mean        std     min     25%     50%     75%     max   
POS                                                                   
1B   1951.066740  32.693286  1876.0  1928.0  1950.0  1975.0  2021.0   
2B   1954.357688  33.200737  1877.0  1928.0  1951.0  1978.0  2022.0   
3B   1954.964085  33.079524  1881.0  1929.0  1953.0  1980.0  2022.0   
C    1960.995757  35.536391  1876.0  1933.0  1962.5  1992.0  2022.0   
OF   1947.993937  32.344317  1872.0  1923.0  1945.0  1969.0  2022.0   
P    1977.072338  29.458177  1881.0  1958.0  1979.0  2002.0  2022.0   
SS   1960.673655  33.669242  1879.0  1934.0  1960.0  1990.0  2021.0   

    deathMonth                                                deathDay  \
         count      mean       std  min  25%  50%   75%   max    count   
POS                                                                      
1B       914.0  6.403720  3.608137  1.0  3.0  6.0  10.0  12.0    914.0   
2B       917.0  6.369684  3.874347  1.0  3.0  6.0  10.0  12.0    917.0   
3B       891.0  6.569024  3.538949  1.0  3.0  6.0  10.0  12.0    891.0   
C       1414.0  6.521216  3.529761  1.0  3.0  6.0  10.0  12.0   1414.0   
OF      2474.0  6.586500  3.501602  1.0  4.0  7.0  10.0  12.0   2474.0   
P      14294.0  6.430740  3.540423  1.0  3.0  6.0  10.0  12.0  14294.0   
SS      1096.0  6.424270  3.481420  1.0  3.0  7.0   9.0  12.0   1096.0   

                                                       weight              \
          mean       std  min  25%   50%   75%   max    count        mean   
POS                                                                         
1B   14.900438  8.934711  1.0  7.0  14.0  23.0  31.0   1142.0  190.253065   
2B   15.158124  9.271


# OPEN TRYOUTS ARE HERE


In [310]:
df_save = df

In [311]:
height_weight = df[['playerID', 'height', 'weight', 'BMI', 'POS', 'WAR', 'WAA']]

In [312]:
height_weight

,playerID,height,weight,BMI,POS,WAR,WAA
0,aardsda01,75.0,215.0,26.87286,P,0.00,0.0000
1,aardsda01,75.0,215.0,26.87286,NaN,-0.04,-0.0400
2,aardsda01,75.0,215.0,26.87286,NaN,0.00,0.0000
3,aardsda01,75.0,215.0,26.87286,NaN,-0.02,-0.0200
4,aardsda01,75.0,215.0,26.87286,NaN,0.00,0.0000
...,...,...,...,...,...,...,...
178146,strasst01,NaN,NaN,NaN,NaN,-0.27,-0.3123
178147,tetreja01,NaN,NaN,NaN,NaN,-0.21,-0.3888
178148,thompma02,NaN,NaN,NaN,NaN,0.11,0.1504
178149,vothau01,NaN,NaN,NaN,NaN,-0.81,-1.2502


In [290]:
height_weight = height_weight.dropna(subset=['BMI'])

In [291]:
pool = height_weight[
    (height_weight.BMI > height_weight.BMI.quantile(0.92))
    & (height_weight.height < height_weight.height.quantile(0.26))]

In [306]:
candidates = pool.sort_values("weight", ascending=False).set_index('POS')

In [309]:
height_weight

,playerID,height,weight,BMI,POS,WAR,WAA
0,aardsda01,75.0,215.0,26.872860,P,0.00,0.00
1,aardsda01,75.0,215.0,26.872860,NaN,-0.04,-0.04
2,aardsda01,75.0,215.0,26.872860,NaN,0.00,0.00
3,aardsda01,75.0,215.0,26.872860,NaN,-0.02,-0.02
4,aardsda01,75.0,215.0,26.872860,NaN,0.00,0.00
...,...,...,...,...,...,...,...
110501,zwilldu01,66.0,160.0,25.824390,NaN,4.14,3.17
110502,zwilldu01,66.0,160.0,25.824390,NaN,-0.67,-0.86
110503,zychto01,75.0,190.0,23.748109,NaN,NaN,NaN
110504,zychto01,75.0,190.0,23.748109,NaN,NaN,NaN


In [308]:
candidates.sort_values('WAR', ascending=False)

,playerID,height,weight,BMI,WAR,WAA
POS,,,,,,
OF,wilsoha01,66.0,190.0,30.666463,7.38,5.24
SS,tejadmi01,69.0,220.0,32.487961,7.37,4.91
NaN,rodriiv01,69.0,205.0,30.272873,6.54,4.26
NaN,segurje01,70.0,220.0,31.566364,6.42,4.41
NaN,rodriiv01,69.0,205.0,30.272873,6.42,4.26
...,...,...,...,...,...,...
NaN,wilsoju99,68.0,190.0,28.889081,NaN,NaN
NaN,mccarjo99,68.0,190.0,28.889081,NaN,NaN
NaN,gautrsi01,68.0,190.0,28.889081,NaN,-0.12


In [ ]:
huskies_P = (
    df_huskies_tryouts_P.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(13, "weight")
)  # 13 heaviest P
huskies_P

In [194]:
pool.groupby()

,height,weight,BMI
count,5289.000000,5289.000000,5289.000000
mean,69.024579,188.852713,27.857111
std,1.290663,13.521389,1.672563
min,55.000000,155.000000,26.544442
25%,68.000000,180.000000,26.581059
50%,69.000000,185.000000,27.319422
75%,70.000000,195.000000,28.409727
max,70.000000,268.000000,38.453570


In [180]:
height_weight.sample(4)

107957    25.423271
77572     23.734004
97324     25.090233
6927      30.430618
Name: BMI, dtype: float64

In [173]:
candidates = df[(df.BMI > df.BMI.quantile(0.75)) & (df.height < df.height.quantile(.25))] # Filter height/weight
candidates

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,KG,meters,BMI,ratio,mlbID,fangraphsID,mlb_played_first,mlb_played_last,name_common_x,age,year_ID,team_ID,stint,lg_ID,PA,G,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_rep,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAA,WAA_off,WAA_def,WAR,WAR_def,WAR_off,WAR_rep,salary,pitcher,teamRpG,oppRpG,oppRpPA_rep,oppRpG_rep,pyth_exponent,pyth_exponent_rep,waa_win_perc,waa_win_perc_off,waa_win_perc_def,waa_win_perc_rep,OPS_plus,TOB_lg,TB_lg,name_common_y,GS_x,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg_adj,RpO_replacement,GR_leverage_index_avg,WAA_adj,ERA_plus,ER_lg,Glf,Gcf,Grf,teamID,lgID,POS,GS_y,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
161,abernbi01,1929.0,1.0,30.0,USA,CA,Torrance,2006.0,2.0,19.0,USA,CA,Yucaipa,Bill,Abernathie,William Edward,190.0,70.0,R,R,1952-09-27,1952-09-27,aberb101,abernbi01,86.182480,1.7780,27.261859,48.471586,110019.0,1000018.0,1952.0,1952.0,Bill Abernathie,23.0,1952.0,CLE,1.0,AL,1.0,1.0,NaN,-0.23,0.00,0.00,0.0,NaN,NaN,NaN,NaN,0.0,0.00,0.11,0.00,-0.1,-0.1,-0.1,0.0,-0.01,-0.01,0.00,-0.01,0.00,-0.01,0.00,NaN,Y,4.04466,4.16466,0.07507,4.13399,1.822,1.828,0.4867,0.4867,0.5000,0.4966,-100.000000,0.330,0.368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLE,AL,P,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
244,abreuto01,1984.0,11.0,13.0,D.R.,Puerto Plata,Puerto Plata,NaN,NaN,NaN,NaN,NaN,NaN,Tony,Abreu,Etanislao Toni,200.0,70.0,B,R,2007-05-22,2014-07-28,abret001,abreuto01,90.718400,1.7780,28.696694,51.022722,473234.0,5053.0,2007.0,2014.0,Tony Abreu,22.0,2007.0,LAD,1.0,NL,178.0,59.0,358.0,-5.14,0.68,-0.03,8.0,0.0,0.0,0.0,0.0,8.0,1.50,0.00,5.55,10.6,5.0,-3.0,9.5,0.48,-0.32,0.94,1.00,0.94,0.20,0.52,NaN,N,4.72784,4.77852,0.08882,4.68448,1.900,1.897,0.5086,0.4949,0.5162,0.4906,82.124031,61.143,72.691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,abreuto01,1984.0,11.0,13.0,D.R.,Puerto Plata,Puerto Plata,NaN,NaN,NaN,NaN,NaN,NaN,Tony,Abreu,Etanislao Toni,200.0,70.0,B,R,2007-05-22,2014-07-28,abret001,abreuto01,90.718400,1.7780,28.696694,51.022722,473234.0,5053.0,2007.0,2014.0,Tony Abreu,24.0,2009.0,LAD,1.0,NL,11.0,6.0,19.0,0.15,-0.44,0.00,2.0,0.0,0.0,0.0,0.0,2.0,0.05,0.00,0.34,2.1,1.8,-0.2,2.1,0.19,-0.03,0.22,0.23,0.22,0.01,0.04,400000.0,N,4.46898,4.50898,0.08274,4.45183,1.869,1.868,0.5313,0.4958,0.5364,0.4940,97.498982,3.652,3.301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,abreuto01,1984.0,11.0,13.0,D.R.,Puerto Plata,Puerto Plata,NaN,NaN,NaN,NaN,NaN,NaN,Tony,Abreu,Etanislao Toni,200.0,70.0,B,R,2007-05-22,2014-07-28,abret001,abreuto01,90.718400,1.7780,28.696694,51.022722,473234.0,5053.0,2007.0,2014.0,Tony Abreu,25.0,2010.0,ARI,1.0,NL,201.0,81.0,338.3,-15.23,-0.13,-1.46,-2.0,0.0,0.0,0.0,0.0,-2.0,1.46,0.00,6.26,-11.1,-17.4,-15.4,-0.5,-1.90,-1.68,-0.09,-1.30,-0.09,-1.08,0.60,407000.0,N,4.19157,4.38120,0.08031,4.30386,1.845,1.852,0.4770,0.4796,0.4993,0.4918,46.968202,68.038,81.388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,abreuto01,1984.0,11.0,13.0,D.R.,Puerto Plata,Puerto Plata,NaN,NaN,NaN,NaN,NaN,NaN,Tony,Abreu,Etanislao Toni,200.0,70.0,B,R,2007-05-22,2014-07-28,abret001,abreuto01,90.718400,1.7780,28.696694,51.022722,473234.0,5053.0,2007.0,2014.0,Tony Abreu,27.0,2012.0,KCR,1.0,AL,74.0,22.

birthYear                                                                  \
        count         mean        std     min     25%     50%     75%     max   
POS                                                                             
C         7.0  1881.285714  45.294276  1864.0  1864.0  1864.0  1864.5  1984.0   
SS        8.0  1974.000000   0.000000  1974.0  1974.0  1974.0  1974.0  1974.0   

    birthMonth                                         birthDay             \
         count mean       std  min  25%  50%  75%  max    count       mean   
POS                                                                          
C          7.0  5.0  1.732051  2.0  4.5  6.0  6.0  6.0      7.0  22.285714   
SS         8.0  5.0  0.000000  5.0  5.0  5.0  5.0  5.0      8.0  25.000000   

                                             deathYear                         \
           std   min   25%   50%   75%   max     count         mean       std   
POS                                                                             
C    11.643514   2.0  19.0  29.0  29.0  29.0       6.0  1936.166667  5.307228   
SS    0.000000  25.0  25.0  25.0  25.0  25.0       0.0          NaN       NaN   

                                            deathMonth                      \
        min     25%     50%     75%     max      count      mean       std   
POS                                                                          
C    1934.0  1934.0  1934.0  1934.0  1947.0        6.0  7.666667  0.816497   
SS      NaN     NaN     NaN     NaN     NaN        0.0       NaN       NaN   

                             deathDay                                          \
     min  25%  50%  75%  max    count      mean       std  min  25%  50%  75%   
POS                                                                             
C    6.0  8.0  8.0  8.0  8.0      6.0  9.666667  4.082483  8.0  8.0  8.0  8.0   
SS   NaN  NaN  NaN  NaN  NaN      0.0       NaN       NaN  NaN  NaN  NaN  NaN   

          weight                                                           \
      max  count        mean       std    min    25%    50%    75%    max   
POS                                                                         
C    18.0    7.0  214.714286  0.755929  213.0  215.0  215.0  215.0  215.0   
SS    NaN    8.0  220.000000  0.000000  220.0  220.0  220.0  220.0  220.0   

    height                                                      KG             \
     count       mean      std   min   25%   50%   75%   max count       mean   
POS                                                                             
C      7.0  68.285714  0.48795  68.0  68.0  68.0  68.5  69.0   7.0  97.392682   
SS     8.0  69.000000  0.00000  69.0  69.0  69.0  69.0  69.0   8.0  99.790240   

                                                                 meters  \
          std        min       25%       50%       75%       max  count   
POS                                                                       
C    0.342883  96.615096  97.52228  97.52228  97.52228  97.52228    7.0   
SS   0.000000  99.790240  99.79024  99.79024  99.79024  99.79024    8.0   

                                                                  BMI  \
         mean       std     min     25%     50%     75%     max count   
POS                                                                     
C    1.734457  0.012394  1.7272  1.7272  1.7272  1.7399  1.7526   7.0   
SS   1.752600  0.000000  1.7526  1.7526  1.7526  1.7526  1.7526   8.0   

                                                                      \
          mean       std        min        25%        50%        75%   
POS                                                                    
C    32.379319  0.537861  31.454253  32.219937  32.690276  32.690276   
SS   32.487961  0.000000  32.487961  32.487961  32.487961  32.487961   

               ratio                                                        \
           max count       mean

In [16]:
df["BMI"] = round(df.BMI, 2)  # clean up our BMI decimals
df = df.dropna(subset=["BMI"])  # Drop anyone without a weight.
df_huskies_tryouts = df[df["weight"] >= 225]

df_huskies_tryouts = df_huskies_tryouts.sort_values("weight", ascending=False)

In [17]:
# weight of our heaviest starting lineup
df_huskies_tryouts.groupby("POS").weight.max().sum()

1892.0

In [18]:
df_huskies_tryouts

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,KG,meters,BMI,ratio,mlbID,fangraphsID,mlb_played_first,mlb_played_last,name_common_x,age,year_ID,team_ID,stint,lg_ID,PA,G,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_rep,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAA,WAA_off,WAA_def,WAR,WAR_def,WAR_off,WAR_rep,salary,pitcher,teamRpG,oppRpG,oppRpPA_rep,oppRpG_rep,pyth_exponent,pyth_exponent_rep,waa_win_perc,waa_win_perc_off,waa_win_perc_def,waa_win_perc_rep,OPS_plus,TOB_lg,TB_lg,name_common_y,GS_x,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg_adj,RpO_replacement,GR_leverage_index_avg,WAA_adj,ERA_plus,ER_lg,Glf,Gcf,Grf,teamID,lgID,POS,GS_y,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
109928,youngwa01,1980.0,2.0,18.0,USA,MS,Hattiesburg,2015.0,9.0,19.0,USA,MS,Purvis,Walter,Young,Walter Ernest,320.0,77.0,L,R,2005-09-06,2005-10-02,younw001,youngwa01,145.14944,1.9558,37.95,74.214869,425488.0,2112.0,2005.0,2005.0,Walter Young,25.0,2005.0,BAL,1.0,AL,37.0,14.0,81.0,0.47,-0.12,0.19,0.0,0.0,0.0,0.0,0.0,0.0,-0.81,0.00,1.39,1.1,-0.3,-0.3,-0.8,-0.03,-0.03,-0.07,0.10,-0.07,0.10,0.13,NaN,N,4.69790,4.71718,0.08197,4.61808,1.895,1.890,0.4981,0.4981,0.4942,0.4900,115.029797,12.192,13.972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25063,diazju03,1984.0,2.0,27.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,NaN,NaN,NaN,Jumbo,Diaz,Jose Rafael,315.0,76.0,R,R,2014-06-20,2017-07-16,diazj005,diazju03,142.88148,1.9304,38.34,74.016515,471822.0,3397.0,2014.0,2017.0,Jumbo Diaz,30.0,2014.0,CIN,1.0,NL,0.0,34.0,34.7,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,-0.03,0.00,-0.03,0.00,0.00,NaN,Y,3.99035,3.99035,0.07088,3.99035,1.808,1.808,0.5000,0.5000,0.5000,0.5000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25064,diazju03,1984.0,2.0,27.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,NaN,NaN,NaN,Jumbo,Diaz,Jose Rafael,315.0,76.0,R,R,2014-06-20,2017-07-16,diazj005,diazju03,142.88148,1.9304,38.34,74.016515,471822.0,3397.0,2014.0,2017.0,Jumbo Diaz,31.0,2015.0,CIN,1.0,NL,0.0,58.0,60.3,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,-0.01,0.00,-0.01,0.00,0.00,510000.0,Y,4.22222,4.22222,0.07613,4.22222,1.837,1.837,0.5000,0.5000,0.5000,0.5000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25065,diazju03,1984.0,2.0,27.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,NaN,NaN,NaN,Jumbo,Diaz,Jose Rafael,315.0,76.0,R,R,2014-06-20,2017-07-16,diazj005,diazju03,142.88148,1.9304,38.34,74.016515,471822.0,3397.0,2014.0,2017.0,Jumbo Diaz,32.0,2016.0,CIN,1.0,NL,1.0,45.0,43.0,-0.28,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.14,0.00,-0.1,-0.1,-0.1,0.0,-0.01,-0.01,-0.02,-0.01,-0.02,-0.01,0.00,525000.0,Y,4.49699,4.50010,0.08241,4.49938,1.870,1.870,0.4997,0.4997,0.5000,0.4999,-100.000000,0.332,0.427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CIN,NL,P,0.0,129.0,1.0,3.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN
25066,diazju03,1984.0,2.0,27.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,NaN,NaN,NaN,Jumbo,Diaz,Jose Rafael,315.0,76.0,R,R,2014-06-20,2017-07-16,diazj005,diazju03,142.88148,1.9304,38.34,74.016515,471822.0,3397.0,2014.0,2017.0,Jumbo Diaz,33.0,2017.0,TBR,1.0,AL,0.0,2.0,30.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0

In [ ]:
# Probably easiest if we make ourselves a dataframe for every position, since we'll need to fill up a 26-man roster, which is parsed into several discrete roles that each require a particular number of players. 

# These role counts aren't codified, but teams have nearly always carried the same distribution of player-roles.

df_huskies_tryouts_P = df_huskies_tryouts[df_huskies_tryouts['POS'] == "P"]
df_huskies_tryouts_C = df_huskies_tryouts[df_huskies_tryouts['POS'] == "C"]
df_huskies_tryouts_1B = df_huskies_tryouts[df_huskies_tryouts['POS'] == "1B"]
df_huskies_tryouts_2B = df_huskies_tryouts[df_huskies_tryouts['POS'] == "2B"]
df_huskies_tryouts_3B = df_huskies_tryouts[df_huskies_tryouts['POS'] == "3B"]
df_huskies_tryouts_SS = df_huskies_tryouts[df_huskies_tryouts['POS'] == "SS"]
df_huskies_tryouts_OF = df_huskies_tryouts[df_huskies_tryouts['POS'] == "OF"]


In [ ]:
df_huskies_tryouts_P

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,KG,meters,BMI,ratio,mlbID,fangraphsID,mlb_played_first,mlb_played_last,name_common_x,age,year_ID,team_ID,stint,lg_ID,PA,G,Inn,runs_bat,runs_br,runs_dp,runs_field,runs_infield,runs_outfield,runs_catcher,runs_good_plays,runs_defense,runs_position,runs_position_p,runs_replacement,runs_above_rep,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAA,WAA_off,WAA_def,WAR,WAR_def,WAR_off,WAR_rep,salary,pitcher,teamRpG,oppRpG,oppRpPA_rep,oppRpG_rep,pyth_exponent,pyth_exponent_rep,waa_win_perc,waa_win_perc_off,waa_win_perc_def,waa_win_perc_rep,OPS_plus,TOB_lg,TB_lg,name_common_y,GS_x,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg_adj,RpO_replacement,GR_leverage_index_avg,WAA_adj,ERA_plus,ER_lg,Glf,Gcf,Grf,teamID,lgID,POS,GS_y,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
25065,diazju03,1984.0,2.0,27.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,NaN,NaN,NaN,Jumbo,Diaz,Jose Rafael,315.0,76.0,R,R,2014-06-20,2017-07-16,diazj005,diazju03,142.88148,1.9304,38.34,74.016515,471822.0,3397.0,2014.0,2017.0,Jumbo Diaz,32.0,2016.0,CIN,1.0,NL,1.0,45.0,43.0,-0.28,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.14,0.0,-0.1,-0.1,-0.1,0.0,-0.01,-0.01,-0.02,-0.01,-0.02,-0.01,0.0,525000.0,Y,4.49699,4.50010,0.08241,4.49938,1.870,1.870,0.4997,0.4997,0.5000,0.4999,-100.000000,0.332,0.427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CIN,NL,P,0.0,129.0,1.0,3.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN
11968,brownju01,1907.0,4.0,30.0,USA,RI,Greene,1966.0,10.0,2.0,USA,NY,Freeport,Jumbo,Brown,Walter George,295.0,76.0,R,R,1925-08-26,1941-08-27,browj110,brownju01,133.80964,1.9304,35.91,69.317053,111597.0,1001533.0,1925.0,1941.0,Jumbo Brown,31.0,1938.0,NYG,1.0,NL,18.0,43.0,NaN,-1.13,-0.52,-0.24,0.0,NaN,NaN,NaN,NaN,0.0,0.00,2.07,0.0,0.2,0.2,0.2,0.0,0.01,0.02,-0.03,0.01,-0.03,0.02,0.0,NaN,Y,4.42789,4.42371,0.07970,4.41010,1.862,1.861,0.5004,0.5004,0.5000,0.4986,45.477489,6.116,6.277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NY1,NL,P,NaN,NaN,5.0,11.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN
11967,brownju01,1907.0,4.0,30.0,USA,RI,Greene,1966.0,10.0,2.0,USA,NY,Freeport,Jumbo,Brown,Walter George,295.0,76.0,R,R,1925-08-26,1941-08-27,browj110,brownju01,133.80964,1.9304,35.91,69.317053,111597.0,1001533.0,1925.0,1941.0,Jumbo Brown,30.0,1937.0,NYG,2.0,NL,0.0,4.0,NaN,0.00,0.00,0.00,0.0,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,NaN,Y,4.55686,4.55686,0.08283,4.55686,1.877,1.877,0.5000,0.5000,0.5000,0.5000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NY1,NL,P,NaN,NaN,0.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
11969,brownju01,1907.0,4.0,30.0,USA,RI,Greene,1966.0,10.0,2.0,USA,NY,Freeport,Jumbo,Brown,Walter George,295.0,76.0,R,R,1925-08-26,1941-08-27,browj110,brownju01,133.80964,1.9304,35.91,69.317053,111597.0,1001533.0,1925.0,1941.0,Jumbo Brown,32.0,1939.0,NYG,1.0,NL,12.0,31.0,NaN,0.28,-0.46,0.17,0.0,NaN,NaN,NaN,NaN,0.0,0.00,1.32,0.0,1.3,1.3,1.3,0.0,0.14,0.14,-0.02,0.14,-0.02,0.14,0.0,7500.0,Y,4.51438,4.47213,0.08040,4.45954,1.870,1.866,0.5044,0.5044,0.5000,0.4987,110.357093,4.168,4.425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NY1,NL,P,NaN,NaN,0.0,12.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN
11970,brownju01,1907.0,4.0,30.0,USA,RI,Greene,1966.0,10.0,2.0,USA,NY,Freeport,Jumbo,Brown,Walter George,295.0,76.0,R,R,1925-08-26,1941-08-27,browj110,brownju01,133.80964,1.9304,35.91,69.317053,111597.0,1001533.0,1925.0,1941.0,Jumbo Brown,33.0,1940.0,NYG,1.0,NL,10.0,41.0,NaN,-1.89,0.00,0.00,0.0,NaN,NaN,NaN,NaN,0.0,0.00,1.10,0.0,-0.8,-0.8,-0.8,0.0,-0.08,-0.08,-0.01,

We'll need:
- 5 starting pitchers
- 7 relievers
- 2 catchers
- 4 starting infielders
- 2 utility infielders
- 5 outfielders


In [ ]:
huskies_P = (
    df_huskies_tryouts_P.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(13, "weight")
)  # 13 heaviest P

In [ ]:
huskies_C = (
    df_huskies_tryouts_C.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(2, "weight")
)  # 2 heaviest catchers

In [ ]:
huskies_1B = (
    df_huskies_tryouts_1B.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(1, "weight")
)  # heaviest 1B

In [ ]:
huskies_2B = (
    df_huskies_tryouts_2B.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(2, "weight")
)  # 2 heaviest 2B

In [ ]:
huskies_3B = (
    df_huskies_tryouts_3B.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(1, "weight")
)  # 1 heaviest 3B

In [ ]:
huskies_SS = (
    df_huskies_tryouts_SS.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(2, "weight")
)  # 2 heaviest SS

In [ ]:
huskies_OF = (
    df_huskies_tryouts_OF.sort_values("weight", ascending=False)
    .drop_duplicates(subset=["playerID"])
    .nlargest(5, "weight")
)  # 5 heaviest OF

In [ ]:
huskies = pd.concat(
    [huskies_P, huskies_C, huskies_1B, huskies_2B, huskies_3B, huskies_SS, huskies_OF],
    ignore_index=True,
)

In [ ]:
# BEHOLD. THE WONKAVILLE HUSKIES.
huskies.POS.value_counts().sum()

26

In [ ]:
df = huskies.dropna(axis=1)

In [ ]:
print(list(df.columns))

['playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry', 'birthCity', 'nameFirst', 'nameLast', 'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame', 'retroID', 'bbrefID', 'KG', 'meters', 'BMI', 'ratio', 'mlbID', 'fangraphsID', 'mlb_played_first', 'mlb_played_last', 'name_common_x', 'age', 'year_ID', 'team_ID', 'stint', 'lg_ID', 'PA', 'G', 'runs_bat', 'runs_br', 'runs_dp', 'runs_field', 'runs_defense', 'runs_position', 'runs_position_p', 'runs_replacement', 'runs_above_rep', 'runs_above_avg', 'runs_above_avg_off', 'runs_above_avg_def', 'WAA', 'WAA_off', 'WAA_def', 'WAR', 'WAR_def', 'WAR_off', 'WAR_rep', 'pitcher', 'teamRpG', 'oppRpG', 'oppRpPA_rep', 'oppRpG_rep', 'pyth_exponent', 'pyth_exponent_rep', 'waa_win_perc', 'waa_win_perc_off', 'waa_win_perc_def', 'waa_win_perc_rep', 'TOB_lg', 'TB_lg', 'teamID', 'lgID', 'POS', 'PO', 'A', 'E', 'DP']


In [ ]:
df.runs_position_p.value_counts()

0.00    18
0.14     2
2.57     1
5.86     1
0.27     1
0.15     1
0.16     1
0.29     1
Name: runs_position_p, dtype: int64

In [ ]:
# check to make sure everyone's best season is in here
# make sure we've got a rotation and relievers
# clean up columns
huskies.weight.describe()


count     26.000000
mean     271.423077
std       18.465477
min      230.000000
25%      260.000000
50%      275.000000
75%      281.500000
max      315.000000
Name: weight, dtype: float64

In [ ]:
roster_of = df_huskies_tryouts_OF.groupby("playerID", as_index=False)["WAR"].max()